In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

## Importing the dataset

In [ ]:
df = pd.read_csv("training.csv")
test_data = pd.read_csv("test_.csv")

test_id = test_data['id']

## Tokenizing the data

In [ ]:
def tokenizing(msg):
    return[i for i in msg.split()]

In [ ]:
df["text"] = df['text'].apply(tokenizing)
test_data['text'] = test_data['text'].apply(tokenizing)

In [ ]:
df['text'][0]

###### Build the model using tensorfow padding and embedding

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df.head()

In [ ]:
test_data.head()

######  Padding the data

In [ ]:
X = pad_sequences(df['text'], maxlen=200)
X_test = pad_sequences(test_data['text'], maxlen=200)


In [ ]:
y = df['category']

##### Spliting the data

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

##### Handling the class imbalance issue

In [ ]:
from imblearn.over_sampling import SMOTE

smt = SMOTE(sampling_strategy = "minority")

X,y = smt.fit_resample(x_train, y_train)

In [ ]:
pd.Series(y).value_counts()

#### Defining the layers

In [ ]:
vocab_size = 9000
embeded_size = 200

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,embeded_size,input_shape=(X.shape[1],)))
model.add(tf.keras.layers.LSTM(units=120,activation='tanh'))
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X, y, epochs = 4, batch_size = 128, validation_data=(x_test, y_test))

In [ ]:
pred = model.predict_classes(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

In [ ]:
final_test = model.predict_classes(X_test)

final_test = pd.DataFrame(final_test)


In [ ]:
final_df = pd.concat([test_id, final_test],axis=1)

In [ ]:
final_df.rename({0:"category"},axis=1,inplace=True)

In [ ]:
final_df['category'].value_counts()

In [ ]:
#final_df.to_csv("submission_1.csv", index=False)